## Face 값 중 너무 큰 값으로 튀는 outlier 제거

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

# 데이터 표준화를 위해 scipy.stats 라이브러리 사용
import matplotlib.pyplot as plt
import scipy.stats as stats

## 이상치 제거 전, 불필요한 열을 제거하자.

In [ ]:
file_list = ["B","C","D","E","F","G"] # A, D, E, F, G, C
step_list = ['1W','2W','3W','4W']
humans = ["B1", "B2", "B3", "B4",
          "C1", "C2", "C3", "C4",
          "D1", "D2", "D3", "D4",
          "E1", "E2", "E3", "E4",
          "F1", "F2", "F3", "F4",
          "G1", "G2", "G3", "G4"] # "A1", "A2", "A3", "A4"

# 'C1', 'C2', 'C3', 'C4'
# 'A1', 'A2', 'A3', 'A4', 'D1', 'D2', 'D3', 'D4',
            #   'E1', 'E2', 'E3', 'E4', 'F1', 'F2', 'F3', 'F4',
            #   'G1', 'G2', 'G3', 'G4'

# 불필요한 열들을 제거한 후, 최종적으로 파일들을 저장해둘 경로.
for out in file_list:
    for steps in step_list:
        filtered_list = [human for human in humans if human.startswith(out)]
        for human in filtered_list:
            out_path = f"D:/MultiModal/filtered_df/{out}/{steps}_{human}/" # 최종경로. 
            #print(out_path)
            if not os.path.exists(out_path):
                os.makedirs(out_path)

# face 파일들이 있는 구간.
for files in file_list:
    default_path = f"D:/MultiModal/MultiModal_Model/results/face/face_landmark/{files}/"
    # D:\MultiModal\MultiModal_Model\results\face\face_landmark\A
    path1 = os.listdir(default_path)
    
    # # path1 안에 들어 있는 '파일 명'만 있는 것이지 그 외는 아님. 
    for file in path1:
        if '.xlsx' in file:
            #print(file) # 파일명 출력해줌,. 
            full_path = os.path.join(default_path, file)
            #print(full_path)
            b_rotation = pd.read_excel(full_path)
            #print(b_rotation)
            filtered_df = b_rotation.drop(['right eye.x', 'right eye.y', 'left eye.x', 'left eye.y', 'nose.x', 'nose.y', 'mouse.x', 'mouse.y', 'right ear.x',
                                        'right ear.y', 'left ear.x', 'left ear.y', 'box_origin.x', 'box_origin.y'], axis = 1)
            
            filtered_df.to_excel(os.path.join(out_path, f"filtered_{file}"), index=False)

## 이상치를 제거 코드 (파일 경로 접근 + 제거)

In [1]:
# outlier를 제거하는 부분 다시 해볼 것.

# df = 데이터 프레임 넣기, column => outlier를 제거하기 위한 열 값 할당. z_score_threshold -> 기준 값 할당.
def correct_outliers(df, column, z_score_threshold=2):
    
    mean = df[column].mean() ## 열 값에 해당하는 평균 
    std = df[column].std() ## 열 값에 해당하는 std 값 구하기 .(std* 표준편차)

    # Calculate the Z-scores
    z_scores = (df[column] - mean) / std  ## 열 값이 평균으로 부터 얼만큼 일탈되었는가 (멀어져 있는가>? )

    # z_score 값이 얼마인지 확인해볼 것. 
    print(z_scores)
    
    # Identify outliers
    outliers = abs(z_scores) > z_score_threshold ## 2보다 크면 많이 평균으로부터 많이 떨어져 있고, 매우 크거나 작다는 것을 의미. 

    # Replace outliers with the median of the column
    median_value = df[column].median() ## 중앙값 구하기. 
    df.loc[outliers, column] = median_value ### df.loc() = median_value를 넣어줌. 

    return df

In [6]:
import re
import math
import os
import pandas as pd

steps = ["1W", "2W", "3W", "4W"] # "2W", "3W", "4W"

humans = ["A1","A2", "A3", "A4"]

# , "A2", "A3", "A4",
#   "B1", "B2", "B3", "B4",
#   "C1", "C2", "C3", "C4",
#   "D1", "D2", "D3", "D4",
#   "E1", "E2", "E3", "E4",
#   "F1", "F2", "F3", "F4",
#   "G1", "G2", "G3", "G4"          


# load_path = "D:/MultiModal/filtered_df/"

# for group in ["A"]:   # "B", "C", "D", "E", "F", "G"
#     for step in steps:
#         filtered_human = [human for human in humans if human.startswith(group)]
#         for human in filtered_human:
#             dir 
#             if os.path.exists(load_path):
#                 os.makedirs(load_path)


for group in ["A"]:   # "B", "C", "D", "E", "F", "G"
    for step in steps:
        filtered_human = [human for human in humans if human.startswith(group)]
        for human in filtered_human:
            ## 경로 수정해줄 것. 
            load_path = f"D:/MultiModal/filtered_df/{group}/{step}_{human}/"
                    
            # 작업 디렉토리 가져오기. 
            file_list = os.listdir(load_path)
            #print(file_list)
            # 출력할 파일 불러오기
            
            output_folder = f"D:/MultiModal/Face_z_score/"
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            
            for file in file_list:
                dir = os.path.join(load_path, file)
                if dir.split('.')[-1] == "xlsx": 
                    face_df = pd.read_excel(dir)

                    # 뽑아내고 싶은 열 이름 모음. 
                    coordinate_columns = ["box.width", "box.height"]

                    for column in coordinate_columns:
                        # 뽑아낼 열 이름 넣어주고, 저장. 
                        filtered_df = correct_outliers(face_df, column)
                        
                        print(filtered_df)
            #             # filtered_df 안에 값을 넣어서, 저장해줌. 
            #         # filterd_df.iloc[:, 1:].to_excel(output_folder + file.split('.')[0] + '.xlsx')

0        2.746141
1        2.671374
2        2.447076
3        2.596608
4        2.148012
           ...   
29995    0.727458
29996    0.727458
29997    0.802224
29998    0.802224
29999    0.951756
Name: box.width, Length: 30000, dtype: float64
       Unnamed: 0  box.width  box.height
0               0        145         182
1               1        145         181
2               2        145         178
3               3        145         180
4               4        145         174
...           ...        ...         ...
29995       29995        155         155
29996       29996        155         155
29997       29997        156         156
29998       29998        156         156
29999       29999        158         158

[30000 rows x 3 columns]
0        2.746365
1        2.671594
2        2.447280
3        2.596822
4        2.148194
           ...   
29995    0.727538
29996    0.727538
29997    0.802309
29998    0.802309
29999    0.951852
Name: box.height, Length: 30000, dtype:

## 아래 코드는 내가 직접 짜본 '데이터 정규화' 코드 || 문제점 : 정규화를 하는 것은 좋은데 값이 일부 날려보냄

In [ ]:
groups = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
steps = ['1W', '2W', '3W', '4W']
humans = ['A1', 'A2', 'A3', 'A4',
          'B1', 'B2', 'B3', 'B4',
          'C1', 'C2', 'C3', 'C4',
          'D1', 'D2', 'D3', 'D4',
          'E1', 'E2', 'E3', 'E4',
          'F1', 'F2', 'F3', 'F4',
          'G1', 'G2', 'G3', 'G4']

# 저장할 수 있는 경로 설정.
for group in groups:
    for step in steps:
        for human in humans:
            result_out_path = f"D:/MultiModal/Face_z_score/{group}/{step}_{human}/"
            if not os.path.exists(result_out_path):
                os.makedirs(result_out_path)

# 경로 설정 
for group in groups:
    for step in steps:
        for human in humans:
            face_file_path = f"D:/MultiModal/filtered_df/{group}/{step}_{human}/"
            
            if os.path.exists(face_file_path):
                files = os.listdir(face_file_path)
                # 엑셀 파일들 리스트 출력하기
                #print(files)     
                
                # files -> listdir()를 통해서 list 형태로 있는 값들을 불러온 것. 
                for i in files:
                    full_path1 = os.path.join(face_file_path, i) # 경로에 있는 값 하나씩 불러오기. 
                    temp_file = pd.read_excel(full_path1) # 하나씩 불러왔으니, 이를 읽기 위해 read_excel 사용. 
                    #print(temp_file)
            
                    # 이상치를 구하기 위한 식) : (값 - 평균) / (표준편차)  => z_score 값
                    outlier_face = temp_file[(abs((temp_file['box.height'] - np.mean(temp_file['box.height'])) / np.std(temp_file['box.height']))) > 1.96].index
                    # 이상치로 판단되는 값을 제거. 
                    new_df = temp_file.drop(outlier_face)
                    #print(new_df)
                    
                    #이상치를 제거했다면, 결과 값을 엑셀에 저장. 
                    new_df.to_excel(os.path.join(result_out_path, f"filtered_outlier_{i}"), index= False)

## 델타 값 계산해서, 그래프 그리기.

In [ ]:
import seaborn as sns

# body_lean_delta 값 계산
body_path = "D:\\MultiModal\\body_lean_all\\F\\"
# B,C,D,E,F,G

# face_lean_delta 값 계산
face_path = "D:/MultiModal/filtered_df/F/"
# B,C,D,E,F,G

out_path = "D:/MultiModal/data_visualize/F/"
if not os.path.exists(out_path):
    os.makedirs(out_path) 

# 각각 뽑아봐야 하는 것들. 
steps = ['4W'] # '2W', '3W', '4W'
humans = ['F1']
# 'A2', 'A3', 'A4'

for step in steps:
    for human in humans:
        dir = os.path.join(face_path, f"{step}_{human}")
        file_list = os.listdir(dir)
        
        dir_body = os.path.join(body_path, f"{step}_{human}")
        file_body_list = os.listdir(dir_body)
        
        for file_b, files in zip(file_body_list, file_list):
            file_zip_b = os.path.join(dir_body, file_b)
            file_dir_b = pd.read_excel(file_zip_b)
            
            # body에서의 각 델타 값을 계산. 
            filtered_delta_b = file_dir_b.diff()
            
            # 그래프를 그리긱 전 NaN 값을 제거. *보간. 
            filtered_delta_b.interpolate(inplace=True)

            # 차이 값이 계산되는지 확인. 
            #print(filtered_delta_b)
            
            file_zip = os.path.join(dir, files)
            read_dir = pd.read_excel(file_zip)
            
            # 불필요한 열 제거. 
            read_dir.drop(['Unnamed: 0'], axis=1, inplace= True)
            
            # 각 델타 값들을 계산
            filtered_delta = read_dir.diff()
            print(filtered_delta) # 비교할 값이 없다면, NaN 값으로 출력. 

            # 그래프를 그리긱 전 NaN 값을 제거. 
            filtered_delta.interpolate(inplace=True)
        
            # 두 개의 데이터 프레임의 최소값을 맞춰주는 것.     
            min_length = min(len(filtered_delta), len(filtered_delta_b))
            filtered_delta = filtered_delta[:min_length]
            filtered_delta_b = filtered_delta_b[:min_length]
    
            # 산점도를 확인해볼 수 있는 것. 
            #lt.scatter(filtered_delta, filtered_delta_b)
            #plt.xlabel('body_lean')  # 바디 기울기.
            #plt.ylabel('face_lean')  # 얼굴 기울기. 
            #plt.title('Scatter Plot of face_diff() vs. body_lean_diff()')
            #plt.grid(True)  # Enable grid
            #plt.show()
        
            new_df =  pd.concat([filtered_delta, filtered_delta_b], axis=1)
            new_df.columns = ['filtered_delta', 'filtered_delta_b']
            
            # 뽑아보고 싶은 특정 열 선택. 
            data_range = new_df.loc[900:1000]
            # Calculate the correlation matrix
            correlation_matrix = new_df.corr()
            plt.figure(figsize=(10, 6))
            
            plt.plot(data_range['filtered_delta'], label='filtered_delta')
            plt.plot(data_range['filtered_delta_b'], label='filtered_delta_b')
            plt.xlabel('Index')
            plt.ylabel('Values')
            plt.title('Graph of face box and body_lean (Index Range: 5100-5450)')
            plt.legend()
            plt.grid(True)
            plt.show()
                    
        
        
            # 히트 맵으로 보여주는 코드 부분
            # 양의 상관 관계는 따뜻한 색상, 음의 상관관계는 파란색으로 나타남.
            # 값이 1또는 -1에 가까울 수록 강한 상관관계를 나타내고, 0에 가까울수록 약한 상관 관계를 나타냄. 
            # Plotting heatmap of correlation matrix
            # plt.figure(figsize=(8, 6))
            # sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
            # plt.title('Correlation Heatmap between filtered_delta and filtered_delta_b')
            # plt.show()
    

## face box scaling 해주는 구간.

In [ ]:
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

for group in ["B", "C", "D", "E", "F", "G"]:
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 
    
    ## D:\MultiModal\MultiModal_postprocessing\results\face\face_landmark\G
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())

    # 출력할 파일 불러오기
    output_folder = f"D:/MultiModal/Face_z_score/Face_lean/{group}/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 컬럼명
    face_landmark_col = ["Week", "ID", "Step", "Face_lean_width", "Face_lean_height"]

    data = []
    for file in file_list:
        if file.split('.')[-1] == "xlsx": # csv 파일만 읽기
            landmark_df = pd.read_excel(file)
            # scaling
            landmark_df["lean_width"] = min_max_scaling(landmark_df["box.width"]) # 너비
            landmark_df["lean_height"] = min_max_scaling(landmark_df["box.height"]) # 높이
            
            file_name = file.split('.')[0].split('_')
            ## + '_' + file_name[-1]
            data.append([file_name[1], file_name[2], file_name[3] if (file_name[3] == "Baseline" or file_name[3] == "S3") else file_name[3], np.mean(landmark_df["lean_width"]), np.mean(landmark_df["lean_height"])])

    avg_box_df = pd.DataFrame(data, columns=face_landmark_col)
    avg_box_df.to_excel(output_folder + f"{group}_average_face_lean.xlsx")

In [ ]:
# all df concat

group_df = []

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    os.chdir(f"D:/MultiModal/Face_z_score/Face_lean/{group}/") 
    ## D:\MultiModal\MultiModal_postprocessing\results\face_time\face_landmark\G
    group_df.append(pd.read_excel(f"{group}_average_face_lean.xlsx").iloc[:, 1:])

    output_folder = f"../"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

all_group_df = pd.concat(group_df)
all_group_df.to_excel(output_folder + "All_average_face_lean(All group, anaysis).xlsx")

In [ ]:
os.chdir("D:/MultiModal/Face_z_score/Face_lean/") 
all_df = pd.read_excel("All_average_face_lean(All group, anaysis).xlsx") 

step_list = ["S1", "S2" "S3"]

df_index = ["Group", "ID"] + [f"{week}_{step}_Face_lean_{option}" for week in ["1W", "2W", "3W", "4W"] for step in step_list for option in ["width", "height"]] 

data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    for id in [f"{group}{id_index}" for id_index in range(1, 5)]:
        week_data = []
        for week in [f"{w_index}W" for w_index in range(1, 5)]:
            filtered_df = all_df[(all_df["Week"] == week) & (all_df["ID"] == id) & (all_df["Group"] == group)]
            print(filtered_df)
            week_data.extend(filtered_df[[f"{step}_Face_lean_{option}" for step in step_list for option in ["width", "height"]]].values.flatten().tolist())
        
        data.append([group, id] + week_data)

# all_summarize_df = pd.DataFrame(data, columns=df_index)
# all_summarize_df.to_excel("All_average_face_lean(All group, excel_anaysis).xlsx")

## 얼굴 기울기 횟수 세는 구간.

In [ ]:
# 얼굴 크기 변화량 값 계산하는 구간. 
def plus_minus(df):
    df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )

    W = "box.width"
    H = "box.height"

    df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
    df_new = df.iloc[1:, :]["box delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 
    
    # df_new 값일 어떻게 생성되는지 확인해보기 위해 출력. 
    #print(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -600):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        else:
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    return df_new 

In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

output_folder = f"D:/MultiModal/Face_z_score/Face_lean_count/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["A","B","C","D","E","F","G"]: # , "B","C", "D", "E", "F", "G"
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 
    #os.chdir(f"D:/MultiModal/MultiModal_postprocessing/results/face_5m/face_landmark/{group}/")

    # 스텝에 나와 있는 것 
    step_list = ["S1", "S2", "S3"]
    #step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

    face_index = ["Group", "Week",] + [f"{step}_Face_lean_3p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        step_data = []
        for step in step_list:
            df1 = pd.read_excel(f'filtered_Face_{week}_{group}1_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'filtered_Face_{week}_{group}2_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'filtered_Face_{week}_{group}3_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'filtered_Face_{week}_{group}4_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}4_{step}.xlsx') else []
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
            #length = a.min()
            length = a.min()-1 # 최솟값에서 -1을 해줌. 

            # # def문을 활용해서 delta 값 만들기. 
            # def plus_minus(df):
            #     df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )
 
            #     W = "box.width"
            #     H = "box.height"

            #     df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
            #     df_new = df.iloc[1:, 18] # ? 왜 열 값을 이렇게 주었을까? 첫 1번 열과 18개의 행만  
                
            #     for i in range(len(df_new)):
            #         if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
            #             df_new.iloc[i] = 1 # 앞으로 기울었을 때 
            #         elif(df_new.iloc[i] < -600):
            #             df_new.iloc[i] = -1 # 뒤로 기울었을 때 
            #         else:
            #             df_new.iloc[i] = 0 # 아무것도 아닐 때
            #     return df_new 
   
            # df1 = plus_minus(df1)
            # print(df1)
   
   
            if len(df1) > 0:
                df1 = plus_minus(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
            
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = 1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            #print(df_concat.head())

            df_concat['same'] = 0 
                        
            for i in range (len(df_concat)):
                
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                df_concat.iloc[i,4] = count.max()
            
            same = np.array(df_concat["same"])
            #step_data.append((df_concat['same'] >= 3).sum())
            step_data.append(np.where(same>=3,1,0).sum())
        
        data.append([group, week] + step_data)
        # 새로운 데이터 프레임을 하나 만들어줄 것. 
        count_df = pd.DataFrame(data, columns=face_index)
    
         # out_folder 위치에 face_lean이 얼마나 기울어졌는지 기록하는 구간.  
        count_df.to_excel(output_folder + "All_Face_lean_count.xlsx")

In [ ]:
# 아래의 경로로 변경
os.chdir("D:/MultiModal/Face_z_score/Face_lean_count/")
all_df = pd.read_excel("All_Face_lean_count.xlsx") 

#step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

step_list = ["S1", "S2", "S3"]

df_index = ["Group"] + [f"{week}_{step}_Face_lean_3p" for week in ["1W", "2W", "3W", "4W"] for step in step_list] 
### , "2W", "3W", "4W"
data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    week_data = []
    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        filtered_df = all_df[(all_df["Week"] == week) & (all_df["Group"] == group)]
        week_data.extend(filtered_df[[f"{step}_Face_lean_3p" for step in step_list]].values.flatten().tolist())
        print(filtered_df)
    
    data.append([group] + week_data)   

all_summarize_df = pd.DataFrame(data, columns=df_index)
all_summarize_df.to_excel("All_Face_lean_count(All group, excel_anaysis).xlsx")

## 몸의 기울기 값 z_score 이상치 제거 코드.

In [ ]:
def correct_outliers(df, column, z_score_threshold=3):
    
    mean = df[column].mean()
    std = df[column].std()

    z_scores = (df[column] - mean) / std
    
    outliers = abs(z_scores) > z_score_threshold

    median_value = df[column].median()
    df.loc[outliers, column] = median_value

    return df

In [ ]:
"""
Body Scaling (이상치 처리) 정리 || 값을 조정해주는 코드
"""
import re
import math

for group in ["A", "B", "C", "D", "E", "F", "G"]:
    os.chdir(f"Users/nabi/Desktop/filtered_df/{group}/")
    
    # 파일 불러오기
    file_list = os.listdir(os.getcwd())
    # 출력할 파일 불러오기
    output_folder = f"Users/nabi/Desktop/filtered_df_result/"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in file_list:
        if file.split('.')[-1] == "xlsx": # xlsx 파일만 읽기
            body_df = pd.read_excel(file)

            coordinate_columns = [col for col in body_df.columns if col.endswith(('.x', '.y', '.z'))]

            for column in coordinate_columns:
                filterd_df = correct_outliers(body_df, column)

            filterd_df.iloc[:, 1:].to_excel(output_folder + file)

## 몸의 기울기 값도 똑같이 얼굴 기울기를 셀 수 있도록 해보자. 

In [ ]:
def plus_minus_1(df):
 
    # body delta라는 이름의 데이터 프레임에 lambda로 조건식 주는 곳
    #df["body delta"] = df["body_lean"].apply(lambda x: 1 if x < 90 else (-1 if x > 90 else 0))
    
    df["body delta"] = df["Body_lean"].diff()
    #print(df["body delta"])
    
    # 첫 번째 열은 비교할 수 있는 값이 없으므로, NaN값이 생길 수 있음. 이를 방지하기 위해 0 값을 할당. 
    df.loc[0, 'Body_lean'] = 0
    df_new = df.iloc[1:, :]["body delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 

    # 절대값을 계산해서 비교.
    #df_diff = abs(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 0.02):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -0.02):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        elif(-0.02 < df_new.iloc[i] < 0.02):
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    
    return df_new 

In [ ]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

# 출력할 수 있는 경로 설정 "D:/MultiModal/body_lean_all/body_lean_count/"
output_folder = f"/Users/nabi/Desktop/body_lean_count/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["A","B","C","D","E","F","G"]: # f"D:/MultiModal/body_lean_all/{group}/"
    os.chdir(f"/Users/nabi/Desktop/filtered_mean_value/{group}/")
            
    #스텝에 나와 있는 것 
    step_list = ["S1", "S2", "S3"]
    body_index = ["Group", "Week",] + [f"{step}_Body_lean_3p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        step_data = []
        for step in step_list: # lean_
            df1 = pd.read_excel(f'Mean_filtered_{week}_{group}1_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'Mean_filtered_{week}_{group}2_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'Mean_filtered_{week}_{group}3_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'Mean_filtered_{week}_{group}4_{step}.xlsx') if os.path.exists(f'Mean_filtered_{week}_{group}4_{step}.xlsx') else []
            
            #print(df1)
            #print(df2)
            #print(df3)
            #print(df4)
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
            
            length = a.min()-1 # 최솟값에서 -1을 해줌. 

            if len(df1) > 0:
                df1 = plus_minus_1(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus_1(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus_1(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus_1(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"body delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = å1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            #print(df_concat.head())

            # same 열 초기화 
            df_concat['same'] = 0 
                        
            for i in range (len(df_concat)):
                #print(i)
                # df_concat.iloc[] 번째의 값을 카운트
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                df_concat.iloc[i,4] = count.max()
            
            same = np.array(df_concat["same"])  
            # step_data 리스트 안에는 same 값이 3보다 크다면 인덱스 값을 반환해서, 만족하면 1 아니면 0을 반환. 
            step_data.append(np.where(same>=3,1,0).sum())
    
        data.append([group, week] + step_data)
        # # 새로운 데이터 프레임을 하나 만들어줄 것. 
        count_df = pd.DataFrame(data, columns=body_index)
    
        # out_folder 위치에 face_lean이 얼마나 기울어졌는지 기록하는 구간.  
        count_df.to_excel(output_folder + "All_Body_lean_count.xlsx")

In [ ]:
# 아래의 경로로 변경
os.chdir("/Users/nabi/Desktop/body_lean_count/") # "D:/MultiModal/body_lean_all/body_lean_count/"
all_df = pd.read_excel("All_Body_lean_count.xlsx") 

#step_list = ["S1_1", "S1_2", "S1_3", "S1_4", "S2_1", "S2_2", "S2_3", "S2_4", "S3"]

step_list = ["S1", "S2", "S3"]

df_index = ["Group"] + [f"{week}_{step}_Body_lean_3p" for week in ["1W", "2W", "3W", "4W"] for step in step_list] 
### , "2W", "3W", "4W"
data = []
for group in ["A", "B", "C", "D", "E", "F", "G"]:
    week_data = []
    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        filtered_df = all_df[(all_df["Week"] == week) & (all_df["Group"] == group)]
        week_data.extend(filtered_df[[f"{step}_Body_lean_3p" for step in step_list]].values.flatten().tolist())
    
    data.append([group] + week_data)   

all_summarize_df = pd.DataFrame(data, columns=df_index)
all_summarize_df.to_excel("All_Body_lean_count(All group, excel_anaysis).xlsx")

## body_lean의 diff() 값을 계산해보자. 

In [ ]:
import numpy as np

def show_the_diff(df):
    # body_diff 라는 이름의 열 하나 생성. || 그리고 body_lean에 대한 diff() * 열 간의 차이 값 계산. 
    df['body diff'] = df['Body_lean'].diff()
    
    # 첫 번째 열은 비교할 수 있는 값이 없으므로, NaN값이 생길 수 있음. 이를 방지하기 위해 0 값을 할당. 
    df.loc[0, 'Body_lean'] = 0
        
    # diff_df 라는 변수 안에 body diff 전체 값을 넣어 줌. 
    diff_df = df.iloc[1:, :]["body diff"]
    
    result_df = np.median(diff_df)
    result = abs(result_df)

    # 마지막 result에는 diff() 값을 반환. 
    return result

In [ ]:
import os
import pandas as pd
# 최종적으로 저장할 경로. 
data = []

for group in ["B"]: 
    os.chdir(f"/Users/nabi/Desktop/filtered_mean_value/{group}/")

    read_dir = pd.read_excel("Mean_filtered_4W_B1_S1.xlsx")
    result_diff = show_the_diff(read_dir)
    
    print(result_diff)

## 인덱스 정보 값을 넣어주기.

In [29]:
import os
import pandas as pd

body_path = r"D:\\MultiModal\\body_lean_all\\G\\"
 
outpath = r"D:\\MultiModal\\body_lean_all\\body_lean_count\\lean(4p_2)\\G\\"
 
 
# 파일 리스트 값을 불러오기. 
list = os.listdir(body_path)

# 리스트 내에 있는 파일 하나씩 읽어오기. 
for file in list:
    read_file = os.path.join(body_path, file)
    read_f = pd.read_excel(read_file)
    
    index_val = []
    
    for i in read_f.index:
        index_val.append(i)
        
    new_df = pd.DataFrame(columns= ["Frame"])
    new_df["Frame"] = index_val
    
    result = pd.concat([new_df, read_f], axis= 1, ignore_index=False)
    result.style.hide(axis='index')
    
    result.to_excel(outpath + f"Add_{file}", index=False)
    #print(result)

## 4명 중에 2명 값 뽑아보기

In [40]:
# 얼굴 크기 변화량 값 계산하는 구간. 
def plus_minus(df):
    df["box size"] = df['box.width'] * df['box.height'] # 박스 사이즈 (얼굴 크기를 이야기 하는 것 )

    W = "box.width"
    H = "box.height"

    df["box delta"] = df["box size"].diff() # 프레임 단위로 박스의 크기의 변화량을 체크
    df_new = df.iloc[1:, :]["box delta"] # 첫 번째 행은 제외, 제외하지 않을 경우, 비교할 값이 없어서 NaN값 생성. 
    
    # df_new 값일 어떻게 생성되는지 확인해보기 위해 출력. 
    #print(df_new)
    
    for i in range(len(df_new)):
        
        if(df_new.iloc[i] > 600):  # 평균적인 사이즈 값
            df_new.iloc[i] = 1 # 앞으로 기울었을 때 
        elif(df_new.iloc[i] < -600):
            df_new.iloc[i] = -1 # 뒤로 기울었을 때 
        else:
            df_new.iloc[i] = 0 # 아무것도 아닐 때
    return df_new 

In [46]:
import os
import pandas as pd
import numpy as np

# 최종적으로 저장할 경로. 
data = []

output_folder = f"D:/MultiModal/Face_z_score/Face_lean_count/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for group in ["A", "B", "C", "D", "E", "F", "G"]: # , "B","C", "D", "E", "F", "G"
    os.chdir(f"D:/MultiModal/Face_z_score/{group}/") 
    #os.chdir(f"D:/MultiModal/MultiModal_postprocessing/results/face_5m/face_landmark/{group}/")

    # 스텝에 나와 있는 것 || 또한 5분씩 짤라서 기록된 부분은 아직 없음. (시도해보면 좋을 듯)
    step_list = ["S1", "S2", "S3"]

    # 3명이 기울었을 때, 2명이 기울었을 때, 1명이 기울었을 때.
    face_index = ["Group", "Week",] + [f"{step}_Face_lean_3p" for step in step_list] + [f"{step}_Face_lean_2p" for step in step_list] + [f"{step}_Face_lean_1p" for step in step_list]

    for week in [f"{w_index}W" for w_index in range(1, 5)]:
        step_data = []
        for step in step_list:
            df1 = pd.read_excel(f'filtered_Face_{week}_{group}1_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}1_{step}.xlsx') else []
            df2 = pd.read_excel(f'filtered_Face_{week}_{group}2_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}2_{step}.xlsx') else []
            df3 = pd.read_excel(f'filtered_Face_{week}_{group}3_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}3_{step}.xlsx') else []
            df4 = pd.read_excel(f'filtered_Face_{week}_{group}4_{step}.xlsx') if os.path.exists(f'filtered_Face_{week}_{group}4_{step}.xlsx') else []
            
            #이상이 따로 없다면, df1~4까지의 len() 길이 값을 받아오고, 데이터 프레임이 비어있는 경우에는 100000 값을 넣어주어 대체함. 
            a = np.array([len(df1) if len(df1) > 0 else 100000, len(df2) if len(df2) > 0 else 100000, len(df3) if len(df3) > 0 else 100000, len(df4) if len(df4) > 0 else 100000])
           
            length = a.min()-1 # 최솟값에서 -1을 해줌. 


            if len(df1) > 0:
                df1 = plus_minus(df1).iloc[:length] # if length > 0 else None
            else: 
                df1 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
        
            if len(df2) > 0:
                df2 = plus_minus(df2).iloc[:length] 
            else:
                df2 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) #{"box delta": [0 for _ in range(length)]}

            if len(df3) > 0:
                df3 = plus_minus(df3).iloc[:length]  
            else:
                df3 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}

            if len(df4) > 0:
                df4 = plus_minus(df4).iloc[:length]
            else:
                df4 = pd.DataFrame({"box delta": [0 for _ in range(length)]}) # {"box delta": [0 for _ in range(length)]}
            
            # 각 그룹별 데이터가 저장되어 있는 데이터 프레임 값 저장. || axis = 1 (세로 (열) 형태로 저장될 수 있도록 함. ) 
            df_concat = pd.concat([df1,df2,df3,df4], axis = 1)
            #print(df_concat.head())

            # 동시에 기울어진 사람들을 카운트 하는 구간. 
            df_concat['same'] = 0 
                        
            for i in range (len(df_concat)):
                
                count=df_concat.iloc[i].value_counts()
                try:
                    count[0] = 0
                except:
                    0
            
                # 4명 중에 i 명이 몇 명인지를 카운트 하는 부분. 
                df_concat.iloc[i,4] = count.max()

            same = np.array(df_concat["same"])
            
            # 동시에 3명이 기울어진다면 이렇게 넣어줄 것. 
            if np.where(same >=3,1,0).any():
                step_data.append(same.sum())
            # 동시에 2명이 앞으로 기울어진다면 
            elif np.where(same >=2,1,0).any():
                step_data.append(same.sum())
            # 동시에 1명이 앞으로 기울어진다면 
            elif np.where(same >= 1,1,0).any():
                step_data.append(same.sum()) 

            # 일반적으로 동시에 3명이 기울이게 된다면, 아래와 같이 기록하면 됨.    
            #step_data.append(np.where(same>=3,1,0).sum())
        
        data.append([group, week] + step_data)
        print(data)
        
        # 새로운 데이터 프레임을 하나 만들어줄 것. 
        # count_df = pd.DataFrame(data, columns=face_index)
    
        #  # out_folder 위치에 face_lean이 얼마나 기울어졌는지 기록하는 구간.  
        # count_df.to_excel(output_folder + "All_Face_lean_count.xlsx")

C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 0 # 아무것도 아닐 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 1 # 앞으로 기울었을 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = -1 # 뒤로 기울었을 때
C:\Users

[['A', '1W']]


C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 0 # 아무것도 아닐 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 1 # 앞으로 기울었을 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = -1 # 뒤로 기울었을 때
C:\Users

[['A', '1W'], ['A', '2W']]


C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 0 # 아무것도 아닐 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 1 # 앞으로 기울었을 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = -1 # 뒤로 기울었을 때
C:\Users

...

[['A', '1W'], ['A', '2W'], ['A', '3W'], ['A', '4W']]


C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 0 # 아무것도 아닐 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 1 # 앞으로 기울었을 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = -1 # 뒤로 기울었을 때
C:\Users

[['A', '1W'], ['A', '2W'], ['A', '3W'], ['A', '4W'], ['B', '1W']]


C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 0 # 아무것도 아닐 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = -1 # 뒤로 기울었을 때
C:\Users\user\AppData\Local\Temp\ipykernel_19336\368210573.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.iloc[i] = 1 # 앞으로 기울었을 때
C:\Users

KeyboardInterrupt: 